# E8 - 프로젝트 - Movielens 영화 추천 실습

## 실습환경 구성 : 작업 디렉토리 생성, 데이터셋 다운로드

* 이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.  
* 이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.  
 - 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.  
 - 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.  
 - 별점을 시청횟수로 해석해서 생각하겠습니다.  
 - 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.  
  
* MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.  
1) wget으로 데이터 다운로드  
wget http://files.grouplens.org/datasets/movielens/ml-1m.zip  
2) 다운받은 데이터를 작업디렉토리로 옮김  
mv ml-1m.zip ~/aiffel/recommendata_iu/data  
3) 작업디렉토리로 이동  
cd ~/aiffel/recommendata_iu/data  
4) 압축 해제  
unzip ml-1m.zip

## 데이터 준비와 전처리

* Movielens 데이터는 ratings.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os

# ratings 파일이 우리가 다룰 데이터 : 파일 경로 저장하고, ratings 목록의 각 행은 '사용자아이디/영화아이디/평점/타임스탬프'
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

# ratings파일(데이터셋) 읽어오기
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 데이터셋에서 평점 3점 이상인 데이터만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

# 원본 데이터 수, 평점 3점 이상인 데이터 수
print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
# 걸러진 후 남은 데이터 비율
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꾸기
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
# 카운트 확인
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 사용하는 컬럼만 남기고 `timestamp` 항목 제거

using_cols = ['user_id', 'movie_id', 'count']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어오기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python',encoding='latin1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


#검색을 쉽게하기 위해 영화제목(title) 문자열을 소문자로 바꿔주기  
movies['title'] = movies['title'].str.lower()  
  
#검색을 쉽게하기 위해 장르(genre) 문자열을 소문자로 바꿔주기  
movies['genre'] = movies['genre'].str.lower()  
  
movies.head(10)

## 데이터 분석

* ratings에 있는 유니크한 영화 개수  
* ratings에 있는 유니크한 사용자 수  
* 가장 인기 있는 영화 30개(인기순)

In [7]:
# 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
# 사용자 수
ratings['user_id'].nunique()

6039

In [9]:
# 인기 있는 영화 30개  
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [10]:
# 유저별 몇 편의 영화를 보는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [11]:
# 유저별 영화평점 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['count'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: count, dtype: float64

## 내가 선호하는 영화 5가지 골라서 rating에 추가

* 우리는 본인의 음악 취향과 가장 유사한 아티스트를 추천받고 싶습니다. 유튜브 뮤직 등 추천시스템들은 이를 위해서 처음 가입하는 사용자의 취향과 유사한 아티스트 정보를 5개 이상 입력받는 과정을 거치게 하는 경우가 많습니다  
* 우리도 동일한 과정이 필요합니다. 우리가 만든 추천시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야 합니다. 저는 아래와 같이 제가 좋아하는 유명한 아티스트들을 기존의 데이터에 추가해서 넣겠습니다.

In [12]:
# 본인이 좋아하시는 영화 데이터 추가 (단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞추기)
my_favorite = ['Toy Story (1995)', 'Sense and Sensibility (1995)', 'While You Were Sleeping (1995)',
               'Saving Grace (2000)', 'Star Wars: Episode I - The Phantom Menace (1999)']
# 메타데이터를 통한 영화 제목별 아이디 확인
my_fav_id = [1, 17, 339, 3831, 2628]

# 10이라는 user_id가 위 영화를 평점 5 줬다고 가정
my_movielist = pd.DataFrame({'user_id': ['10']*5, 'movie_id':my_fav_id,
                             #'title':my_favorite,
                             'count':[5]*5})

if not ratings.isin({'user_id':['10']})['user_id'].any():  # user_id에 '10'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                  # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
ratings.tail(10)     # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,10,1,5
1,10,17,5
2,10,339,5
3,10,3831,5
4,10,2628,5


## 모델에 활용하기 위한 전처리

* 사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶습니다. 우리가 다루는 데이터에서는 user와 movie 각각에 번호를 붙이고 싶습니다. 보통 이런 작업을 indexing이라고 합니다. 추천시스템, 자연어처리에서 자주 하는 작업들이라 직접 해보시면 좋겠습니다.  
  
* (참고) pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용합니다.

In [13]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['10'])    # 6040명의 유저 중 마지막으로 추가된 유저, 6039가 나와야 함  
print(movie_to_idx[339])

6039
579


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,40,5
1,6039,357,5
2,6039,579,5
3,6039,1465,5


* data의 user_id와 movie_id 컬럼 내 값들이 모두 정수 인덱스 값으로 잘 변경되었나요? 이것으로 훈련을 위한 전처리가 완료!!

## 사용자의 명시적/암묵적 평가

In [16]:
# 평점 3이하인 데이터의 비율을 보는 코드
only_one = ratings[ratings['count']<4]
one, all_ratings = len(only_one), len(ratings)
print(f'{one},{all_ratings}')
print(f'Ratio of only_one over all data is {one/all_ratings:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

261197,836483
Ratio of only_one over all data is 31.23%


* 우리가 앞으로 만들어갈 모델에서는 암묵적 데이터의 해석을 위해 다음과 같은 규칙을 적용할 것입니다.  
  
 - 평점 4 이상이면 선호한다고 판단한다.  
 - 평점 높은 영화에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

## CSR(Compressed Sparse Row) Matrix

* 이제 우리의 data를 CSR Matrix에 맞게 바꿔보겠습니다.  
* DataFrame을 만드는 방식이 다양하듯 csr_matrix를 만드는 방법은 다양하게 있습니다.  
* 현재 data와 같은 데이터 구조에 적합한 방식은 위 링크에서 4번째로 설명하고 있는 방법입니다.  
  
* https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html  
* 링크의 네 번째 방법 이용할 것임

In [17]:
print(ratings['count'])

0    5
1    3
2    3
3    4
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: count, Length: 836483, dtype: int64


In [18]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

* 이전 스텝에서 설명한 Matrix Factorization 모델을 implicit 패키지를 사용하여 학습해 봅시다.   
  
 - implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.  
 - 이 패키지에 구현된 als(AlternatingLeastSquares) 모델을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적인 것으로 알려져 있습니다.  
 - https://github.com/benfred/implicit

In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

* AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다. 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 3. use_gpu : GPU를 사용할 것인지 4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지  
  
* 1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [21]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [23]:
# 모델이 1)나의 벡터와 영화 saving grace 벡터를 어떻게 만들고 있는지와 2)두 벡터를 곱하면 어떤 값이 나오는지 살펴보기
# 339 = 'Saving Grace (2000)'
ten, grace = user_to_idx['10'], movie_to_idx[339]
ten_vector, grace_vector = als_model.user_factors[10], als_model.item_factors[339]

print('슝=3')

슝=3


In [24]:
ten_vector

array([ 1.1314503e+00,  1.0161070e+00, -1.9969987e+00, -5.8515835e-01,
        4.6022072e-01, -2.1458943e+00,  3.7225851e-01,  2.4758930e+00,
        2.1071386e+00,  3.4331575e-01,  6.5261686e-01,  3.4106007e-01,
        1.2347453e+00,  1.7085338e-01, -9.8708385e-01, -1.9961222e+00,
       -8.8191766e-01, -1.1113684e+00, -2.0757337e+00, -7.5449985e-01,
        1.4942832e+00, -8.5799986e-01,  8.1136709e-01, -1.4797254e+00,
        8.7067795e-01,  1.3616508e+00, -8.4276700e-01, -1.6491804e+00,
       -9.9610579e-01,  8.1639856e-01,  2.9036472e+00,  2.1148078e+00,
       -2.2510222e-01,  2.2427154e+00,  5.8783251e-01, -2.9964580e-03,
       -1.3371786e-01,  1.5724549e+00,  4.9850170e-02, -3.3281386e+00,
        6.0554785e-01,  1.4568771e+00,  3.2939596e+00,  1.5398943e-01,
        3.1636569e+00, -6.1188799e-01, -1.1386099e+00, -6.6071129e-01,
       -1.9519523e+00, -5.3980511e-01, -3.3430165e-01, -8.1785995e-01,
        1.8323581e+00, -2.0226228e+00,  8.9698231e-01,  1.6134181e+00,
      

In [25]:
grace_vector

array([ 3.0338038e-03,  2.5449935e-02, -5.7756918e-04, -3.3490423e-03,
        9.9686896e-03, -7.8189932e-03,  1.5594130e-03,  8.3303507e-03,
       -2.6470751e-03,  9.4335536e-03, -7.5287898e-03,  2.7508590e-02,
        7.3310719e-03,  4.1093007e-03,  7.7719842e-03, -1.2534331e-02,
       -4.3188709e-05,  1.4656028e-02,  1.2512393e-02, -9.1429253e-04,
        1.1286219e-02, -3.1928492e-03, -7.2982704e-04, -5.8874553e-03,
       -6.3298238e-03,  4.2037450e-02, -6.0970355e-03,  1.4224713e-02,
        1.0486973e-02, -1.2694990e-02,  1.6633805e-02,  1.5170000e-02,
        1.2730195e-02, -2.2375241e-03,  4.1367169e-03,  9.1444058e-03,
        3.6429556e-03,  1.1484757e-02,  3.0660743e-02,  8.6942557e-03,
        9.9988598e-03,  2.8726803e-02,  3.8546398e-03, -1.9448854e-03,
        4.5194590e-04,  9.3369298e-03,  1.2636320e-02,  2.4388796e-02,
        1.7049735e-02,  9.5349550e-03, -2.6522882e-02,  4.5482735e-03,
       -5.7935594e-03,  1.1460101e-02,  7.7746431e-03, -6.3671456e-03,
      

In [26]:
# ten과 grace를 내적하는 코드
np.dot(ten_vector, grace_vector)

0.11253947

In [27]:
# 모델이 나의 '356::Forrest Gump (1994)::Comedy|Romance|War'에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다

forrest = movie_to_idx[356]
forrest_vector = als_model.item_factors[356]
np.dot(ten_vector, forrest_vector)

0.26872516

## 비슷한 아티스트 찾기 + 유저에게 추천하기

### 비슷한 아티스트 찾기

In [28]:
# AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 선호하는 영화를 찾습니다. 
# 처음으로는 제가 좋아하는 '1::Toy Story (1995)::Animation|Children's|Comedy'로 찾아보겠습니다.

favorite_movie = 'Toy Story (1995)'
fav_movie_id = 1
movie_id = movie_to_idx[fav_movie_id]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

# (movie id, 유사도) Tuple 로 반환하고 있습니다.

[(40, 1.0),
 (50, 0.8120418),
 (4, 0.6002671),
 (322, 0.5919847),
 (33, 0.5816359),
 (110, 0.5703447),
 (330, 0.46708557),
 (20, 0.45447257),
 (255, 0.43235815),
 (10, 0.4315646),
 (34, 0.39919424),
 (16, 0.38869423),
 (126, 0.38753217),
 (32, 0.3716275),
 (160, 0.35738182)]

In [29]:
# movie idx를 다시 movie_id로 매핑 시키기
# movie_to_idx 를 뒤집어, index로부터 movie_id를 얻는 dict를 생성합니다. 

idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[1,
 3114,
 2355,
 34,
 588,
 1265,
 364,
 2321,
 1923,
 595,
 1907,
 2687,
 2396,
 1566,
 356]

In [30]:
# movies.dat 파일에서 영화아이디로 제목 불러오기  
sim_movie_id = [idx_to_movie[i[0]] for i in similar_movie]
sim_movie_title = movies.loc[sim_movie_id, :]
sim_movie_title.head(15)

,movie_id,title,genre
1,2,Jumanji (1995),Adventure|Children's|Fantasy
3114,3183,"Third Miracle, The (1999)",Drama
2355,2424,You've Got Mail (1998),Comedy|Romance
34,35,Carrington (1995),Drama|Romance
588,592,Batman (1989),Action|Adventure|Crime|Drama
1265,1285,Heathers (1989),Comedy
364,368,Maverick (1994),Action|Comedy|Western
2321,2390,Little Voice (1998),Comedy
1923,1992,Child's Play 2 (1990),Horror
595,599,"Wild Bunch, The (1969)",Western


In [31]:
# 몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인해보겠습니다.

def get_similar_movie(movie_id: int):
    movie_idx = movie_to_idx[movie_id]
    similar_movie = als_model.similar_items(movie_idx)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [32]:
# '1580::Men in Black (1997)::Action|Adventure|Comedy|Sci-Fi'으로 확인

get_similar_movie(1580)
get_sim_movie_title = movies.loc[get_similar_movie(1580), :]
get_sim_movie_title.head(10)

,movie_id,title,genre
1580,1623,Wishmaster (1997),Horror
480,484,Lassie (1994),Adventure|Children's
589,593,"Silence of the Lambs, The (1991)",Drama|Thriller
2916,2985,Robocop (1987),Action|Crime|Sci-Fi
780,790,An Unforgettable Summer (1994),Drama
2571,2640,Superman (1978),Action|Adventure|Sci-Fi
1527,1567,"Last Time I Committed Suicide, The (1997)",Drama
1544,1585,Love Serenade (1996),Comedy
2657,2726,"Killing, The (1956)",Crime|Film-Noir
989,1002,Ed's Next Move (1996),Comedy


In [33]:
# '1409::Michael (1996)::Comedy|Romance' 확인

get_similar_movie(1409)
get_sim_movie_title = movies.loc[get_similar_movie(1409), :]
get_sim_movie_title.head(10)

,movie_id,title,genre
1409,1433,"Machine, The (1994)",Comedy|Horror
2266,2335,"Waterboy, The (1998)",Comedy
237,240,Hideaway (1995),Thriller
276,279,My Family (1995),Drama
236,239,"Goofy Movie, A (1995)",Animation|Children's|Comedy|Romance
1835,1904,Henry Fool (1997),Comedy|Drama
802,812,Magic Hunter (1994),Drama
252,255,"Jerky Boys, The (1994)",Comedy
339,343,"Baby-Sitters Club, The (1995)",Children's
7,8,Tom and Huck (1995),Adventure|Children's


### 유저에게 영화 추천하기

* AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 내가 좋아할 만한 영화를 추천받기  
* filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [34]:
user = user_to_idx['10']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(50, 0.4459266),
 (322, 0.34380716),
 (64, 0.2861888),
 (670, 0.27354598),
 (38, 0.27284706),
 (4, 0.27000844),
 (352, 0.24594934),
 (626, 0.23803964),
 (44, 0.21869494),
 (117, 0.21611829),
 (1200, 0.20059976),
 (340, 0.19611533),
 (666, 0.19483006),
 (110, 0.19411363),
 (458, 0.19186753),
 (126, 0.18567234),
 (27, 0.18549138),
 (336, 0.18436319),
 (160, 0.17392384),
 (175, 0.1727199)]

In [35]:
r = [idx_to_movie[i[0]] for i in movie_recommended]

get_sim_movie_title = movies.loc[r, :]
get_sim_movie_title.head(10)

# '3114::Toy Story 2 (1999)::Animation|Children's|Comedy' 추천
# '1210::Star Wars: Episode VI - Return of the Jedi (1983)::Action|Adventure|Romance|Sci-Fi|War' 추천
# '2396::Shakespeare in Love (1998)::Comedy|Romance' 추천
# '260::Star Wars: Episode IV - A New Hope (1977)::Action|Adventure|Fantasy|Sci-Fi' 추천
# '2424::You've Got Mail (1998)::Comedy|Romance' 추천

,movie_id,title,genre
3114,3183,"Third Miracle, The (1999)",Drama
34,35,Carrington (1995),Drama|Romance
1210,1228,Raging Bull (1980),Drama
3175,3244,"Goodbye Girl, The (1977)",Comedy|Romance
2762,2831,"Dog of Flanders, A (1999)",Drama
2355,2424,You've Got Mail (1998),Comedy|Romance
838,849,Escape from L.A. (1996),Action|Adventure|Sci-Fi|Thriller
2424,2493,"Harmonists, The (1997)",Drama
260,263,Ladybird Ladybird (1994),Drama
1196,1214,Alien (1979),Action|Horror|Sci-Fi|Thriller


In [36]:
# 모델은 왜 '토이스토리2'를 추천해줬을까? 
# AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 내가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인 가능

toystory2 = movie_to_idx[3114]
explain = als_model.explain(user, csr_data, itemid=3114)

# 이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환

In [37]:
# 어떤 영화들이 이 추천에 얼마나 기여하고 있는 걸까요?

[[idx_to_movie[i[0]], i[1]] for i in explain[1]]

get_sim_movie_title = movies.loc[[17,3831,1,339,2628], :]
get_sim_movie_title.head(5)

# my_favorite = ['Toy Story (1995)', 'Sense and Sensibility (1995)', 'While You Were Sleeping (1995)',
#               'Saving Grace (2000)', 'Star Wars: Episode I - The Phantom Menace (1999)']
# my_fav_id = [1, 17, 339, 3831, 2628]

# 'sense and sensibility'와 'toy story'가 가장 크게 기여 

,movie_id,title,genre
17,18,Four Rooms (1995),Thriller
3831,3901,Duets (2000),Comedy|Drama
1,2,Jumanji (1995),Adventure|Children's|Fantasy
339,343,"Baby-Sitters Club, The (1995)",Children's
2628,2697,My Son the Fanatic (1998),Comedy|Drama|Romance


## 회고

매번 코딩 실력이 부족하여 답답함을 느낀다. 좀 더 효과적으로 데이터을 읽어오는 방법들이 있을 것 같은데 시간에 쫓겨 하다 보니 검색도 안 되고 읽어봐도 무슨 소린지 이해를 못하고 넘어간다. 뭐든 하루아침에 되는 건 없고 그냥 답답함을 조금씩 견디며 차근차근 공부해 나가야 겠다.  
  
* 하고 싶었던 것 : 데이터 병합 (merge 이용하여 ratings.dat와 movies.dat의 필요한 colume)